In [1]:
# Run this code if you get errors with missing requirements
import sys
!{sys.executable} -m pip install mysql-connector

import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="iiwari-mariadb-server",
  user="root",
  passwd="d41k4Duu",
  database="iiwari_org"
)


In [2]:
# Check database contents
mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT node_id, timestamp, x,y,z,q FROM SensorData WHERE node_id = 54016 AND timestamp > '2019-03-20 09:00:00.000000+00:00' AND timestamp < '2019-03-21 22:05:08.078133+00:00' LIMIT 20000 " );
df = pd.DataFrame(mycursor.fetchall())

print(df)

      node_id                            timestamp   x     y  z    q
0       54016  b'2019-03-21 07:54:39.687645+00:00' -50    50  0  233
1       54016  b'2019-03-21 07:54:40.686633+00:00' -50    50  0   76
2       54016  b'2019-03-21 07:54:41.687915+00:00' -50    50  0   99
3       54016  b'2019-03-21 07:54:43.687457+00:00' -50    50  0   96
4       54016  b'2019-03-21 07:54:44.687414+00:00' -50    50  0   32
...       ...                                  ...  ..   ... ..  ...
6313    54016  b'2019-03-21 11:52:47.687037+00:00' -50  2797  0  149
6314    54016  b'2019-03-21 11:52:48.687442+00:00' -50  2846  0  227
6315    54016  b'2019-03-21 11:52:49.687187+00:00' -50  2887  0  157
6316    54016  b'2019-03-21 11:52:50.687142+00:00' -50  2920  0  143
6317    54016  b'2019-03-21 11:52:51.691455+00:00' -50  2946  0   97

[6318 rows x 6 columns]


# Luodaat uusi taulu johon laitetaan saatu data


In [5]:
table = pd.DataFrame()
table['node_id'] = df.node_id
table['T'] = df.timestamp
table['X'] = df.x
table['Y'] = df.y
table['Z'] = df.z
table['Q'] = df.q

print(table)

      node_id                                    T   X     Y  Z    Q
0       54016  b'2019-03-21 07:54:39.687645+00:00' -50    50  0  233
1       54016  b'2019-03-21 07:54:40.686633+00:00' -50    50  0   76
2       54016  b'2019-03-21 07:54:41.687915+00:00' -50    50  0   99
3       54016  b'2019-03-21 07:54:43.687457+00:00' -50    50  0   96
4       54016  b'2019-03-21 07:54:44.687414+00:00' -50    50  0   32
...       ...                                  ...  ..   ... ..  ...
6313    54016  b'2019-03-21 11:52:47.687037+00:00' -50  2797  0  149
6314    54016  b'2019-03-21 11:52:48.687442+00:00' -50  2846  0  227
6315    54016  b'2019-03-21 11:52:49.687187+00:00' -50  2887  0  157
6316    54016  b'2019-03-21 11:52:50.687142+00:00' -50  2920  0  143
6317    54016  b'2019-03-21 11:52:51.691455+00:00' -50  2946  0   97

[6318 rows x 6 columns]


# Leikataan timestampeistä pois turhat rojut

In [6]:
table['T'] = table['T'].astype(str)
table['T'].dtype
table['T'] = table['T'].str.slice(2, -7)
print(table['T'])


0       2019-03-21 07:54:39.687645
1       2019-03-21 07:54:40.686633
2       2019-03-21 07:54:41.687915
3       2019-03-21 07:54:43.687457
4       2019-03-21 07:54:44.687414
                   ...            
6313    2019-03-21 11:52:47.687037
6314    2019-03-21 11:52:48.687442
6315    2019-03-21 11:52:49.687187
6316    2019-03-21 11:52:50.687142
6317    2019-03-21 11:52:51.691455
Name: T, Length: 6318, dtype: object


# Muutetaan muotoon datetime64 ja pyöristetään sekunnin tarkkuudelle

In [7]:
table['T'] = table['T'].astype('datetime64[ns]')
table['T'] = pd.Series(table['T']).dt.round("S")
table['T'].head()

0   2019-03-21 07:54:40
1   2019-03-21 07:54:41
2   2019-03-21 07:54:42
3   2019-03-21 07:54:44
4   2019-03-21 07:54:45
Name: T, dtype: datetime64[ns]

# Poistetaan ylirajojen menevät x,y koordinaatit sekä kaikki jossa Q on on yli 100

In [8]:
table = table[table.Y > -1]
table = table[table.Y < 10407]
table = table[table.Q < 100]
table = table[table.X < 12000]
table = table[table.X > 100]
print(table)

      node_id                   T    X     Y  Z   Q
43      54016 2019-03-21 07:55:23  132  2801  0  61
44      54016 2019-03-21 07:55:23  132  2801  0  61
45      54016 2019-03-21 07:55:24  252  2802  0  91
46      54016 2019-03-21 07:55:25  294  2789  0  70
47      54016 2019-03-21 07:55:26  400  2797  0  90
...       ...                 ...  ...   ... ..  ..
6221    54016 2019-03-21 11:51:17  115  2580  0  85
6222    54016 2019-03-21 11:51:18  117  2585  0  88
6223    54016 2019-03-21 11:51:19  112  2580  0  69
6224    54016 2019-03-21 11:51:20  106  2578  0  71
6225    54016 2019-03-21 11:51:21  103  2574  0  72

[5248 rows x 6 columns]
